In [1]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [2]:
engine = create_engine('sqlite:///real_estate.db')

In [3]:
# 행정동 법정동 코드 불러오기
import PublicDataReader as pdr

hdang_code = pdr.code_hdong() # 행정동코드
bdong_code = pdr.code_bdong() # 법정동코드
conn_code= pdr.code_hdong_bdong()

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.2.1. 시행)
출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.2.1. 시행)
출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.2.1. 시행)


In [27]:
conn_code[]

,시도코드,시도명,시군구코드,시군구명,행정동코드,읍면동명,법정동코드,동리명,생성일자,말소일자
0,11,서울특별시,11000,,1100000000,,1100000000,서울특별시,19880423,
1,11,서울특별시,11110,종로구,1111000000,,1111000000,종로구,19880423,
2,11,서울특별시,11110,종로구,1111000000,,1111090100,창신1동,19880423,19880423
3,11,서울특별시,11110,종로구,1111000000,,1111090200,창신2동,19880423,19880423
4,11,서울특별시,11110,종로구,1111000000,,1111090300,창신3동,19880423,19880423
...,...,...,...,...,...,...,...,...,...,...
55172,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042024,대리,20240118,
55173,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042025,거륜리,20240118,
55174,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042026,식도리,20240118,
55175,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042027,상왕등리,20240118,


In [2]:

engine = create_engine('sqlite:///real_estate.db')
conn_code.to_sql('conn_code', engine, if_exists='replace')

engine.dispose()

NameError: name 'conn_code' is not defined

In [7]:
query = '''
SELECT 법정동코드 apt_extended_code LIMIT 10
'''
#query = '''
#CREATE TABLE apt_extended_code AS
#SELECT *, 법정동시군구코드 || 법정동읍면동코드 AS 법정동코드
#FROM apt_extended;
#'''
#UPDATE apt_extended
#SET 법정동코드 = COALESCE(법정동시군구코드, '') || COALESCE(법정동읍면동코드, '');
#'''
#query = '''
#    SELECT 
#        (COUNT(*) - COUNT(법정동시군구코드)) AS 법정동시군구코드_Null_Count,
#        (COUNT(*) - COUNT(법정동읍면동코드)) AS 법정동읍면동코드_Null_Count
#    FROM apt_extended;
#'''
#query = "SELECT 법정동읍면동코드|| 법정동시군구코드 FROM apt_extended LIMIT 10"

#query = '''ALTER TABLE apt_extended ALTER COLUMN 법정동코드 TYPE TEXT " (sqlite에서 칼럼 타입변경 안됨)

#query = "PRAGMA table_info('apt_extended')"

#query = '''
#SELECT a.*, c.행정동코드
#FROM apt_extended as a
#JOIN conn_code c ON a.법정동코드 = c.법정동코드
#'''
#query = '''
#UPDATE apt_extended 
#SET 법정동코드 = 법정동시군구코드 || 법정동읍면동코드;
#'''
#query = "ALTER TABLE apt_extended ADD 법정동코드 TEXT"
#query = "CREATE TABLE apt_extended AS SELECT * FROM apt"
#query = "DROP TABLE IF EXISTS APT_EXTENDED"
#query = "SELECT COUNT(*) FROM apt_extended"

engine = create_engine('sqlite:///real_estate.db')
with engine.connect() as conn:
    result = conn.execute(text(query))
    for row in result :
        print(row)

OperationalError: (sqlite3.OperationalError) no such column: 법정동코드
[SQL: 
SELECT 법정동코드 apt_extended_code LIMIT 10
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [5]:
from sqlalchemy import MetaData , Table
meta = MetaData()
apt_extended = Table('apt_extended', meta, autoload_with=engine)

apt_extended.drop(engine)

In [19]:
query = "SELECT COUNT(*) FROM apt_coded WHERE 년 = 2015"
#query = "DROP TABLE IF EXISTS apt_coded"

with engine.connect() as connection:
    result = connection.execute(text(query))
    for row in result:
       print(row)

(0,)


In [7]:
engine = create_engine('sqlite:///real_estate.db')

query = '''
CREATE TABLE apt_distinct_coded_temp AS
SELECT s.*, c.행정동코드
FROM (
    SELECT *, 법정동시군구코드 || 법정동읍면동코드 AS 법정동코드
    FROM apt_distinct_temp2
) s
LEFT JOIN (
    SELECT * FROM conn_code_now
    )
    c ON s.법정동코드 = c.법정동코드
'''

query = "SELECT COUNT(*) FROM apt_distinct_coded_temp"

with engine.connect() as connection:
    result = connection.execute(text(query))
    for row in result:
        print(row)

(9712311,)


In [8]:
from sqlalchemy import MetaData , Table
from sqlalchemy.sql import text


engine = create_engine('sqlite:///real_estate.db')
meta = MetaData()

apt = Table('apt', meta, autoload_with=engine)
apt_extended_test = Table('apt_extended', meta, extend_existing=True)
meta.reflect(engine, only=['apt'])

# apt_extended에 행정동코드 칼럼 추가
from sqlalchemy import Column, String
apt_extended_test.append_column(Column('행정동코드', String))

apt_extended.create(engine)




with engine.connect() as connection:
    result = connection.execute(text(query))
    for row in result:
        print(row)
        # Row 객체를 딕셔너리로 변환
        #insert_statement = apt_extended.insert().values(**dict(row))
        #connection.execute(insert_statement)

OperationalError: (sqlite3.OperationalError) table apt_extended already exists
[SQL: 
CREATE TABLE apt_extended (
	"행정동코드" VARCHAR
)

]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [14]:
# 엑셀로 옮기기
import pandas as pd
engine = create_engine("sqlite:///real_estate.db")
for year in range(2006, 2025):
    for month in range(1,13):
        df = pd.read_sql(f'SELECT * FROM apt_distinct_coded_temp WHERE 년 = {year} AND 월 = {month}', con=engine)
        df.to_excel(f'/Users/hj/Dropbox/KU/real_estate/data/apt_coded_{year}_{month}.xlsx', index=False)

KeyboardInterrupt: 